In [24]:
import os
import re
import time
from bs4 import BeautifulSoup
import requests
import json
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import random
import pandas as pd

In [32]:
### 辅助函数
# 获取当前页面中的样点文本信息
def get_point_text_info(driver,data:list):
    """获取当前页面的样点文本信息"""
    # page_source 是您从selenium获取到的页面源代码
    page_source = driver.page_source
    # 使用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(page_source, 'html.parser')
    # 数据行都在一定的HTML结构内，例如在某个table或特定标签内
    for row in soup.find_all('tr'):  # 假设每行数据都在一个<tr>标签内
        # 通过选择class为ant-table-cell的<td>标签，并限制结果数量为17来获取前17个信息
        cols = row.find_all('td', class_='ant-table-cell', limit=17)
        if len(cols) == 17:  # 确保提取到了全部17个所需的列
            data_row = {
                '样点编号': cols[0].text.strip(),
                '坐落单位名称': cols[1].text.strip(),
                '样点类别': cols[2].text.strip(),
                '采样类型': cols[3].text.strip(),
                '土壤类型': cols[4].text.strip(), 
                '土地利用类型': cols[5].text.strip(),
                '是否水团样': cols[6].text.strip(),
                '是否盐碱地': cols[7].text.strip(),
                '布设位置经度': cols[8].text.strip(),
                '布设位置纬度': cols[9].text.strip(),
                '采样位置经度': cols[10].text.strip(),
                '采样位置纬度': cols[11].text.strip(),
                '是否修正': cols[12].text.strip(),
                '当前节点': cols[13].text.strip(),
                '调查时间': cols[14].text.strip(),
                '调查单位': cols[15].text.strip(),
                '调查人': cols[16].text.strip(),
                # '操作' 列被我们有意地忽略了，因为它是“详情”按钮
            }
            data.append(data_row)

# 跳转到下一页
def step_next_page(driver):
    """跳转下一页"""
    # 使用类名和属性选择器
    next_page_button = driver.find_element(By.CSS_SELECTOR, "li.ant-pagination-next[title='下一页']")
    next_page_button.click()
    # # 使用XPath
    # next_page_button = driver.find_element(By.XPATH, "//li[@class='ant-pagination-next' and @title='下一页']")
    # next_page_button.click()
    # # 使用类名
    # next_page_button = driver.find_element(By.CLASS_NAME, "ant-pagination-next")
    # next_page_button.click()




In [11]:
# 配置浏览器
# chrome.exe --remote-debugging-port=9999 --user-data-dir="D:\Selenium\AutomationProfile"
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "localhost:9999")
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(10)

In [30]:
for i in range(42):
    step_next_page(driver)
    time.sleep(1)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=120.0.6099.225)
Stacktrace:
	GetHandleVerifier [0x00007FF78E562142+3514994]
	(No symbol) [0x00007FF78E180CE2]
	(No symbol) [0x00007FF78E0274C3]
	(No symbol) [0x00007FF78E072D29]
	(No symbol) [0x00007FF78E066A0F]
	(No symbol) [0x00007FF78E095FEA]
	(No symbol) [0x00007FF78E0663B6]
	(No symbol) [0x00007FF78E096490]
	(No symbol) [0x00007FF78E0B28F6]
	(No symbol) [0x00007FF78E095D93]
	(No symbol) [0x00007FF78E064BDC]
	(No symbol) [0x00007FF78E065C64]
	GetHandleVerifier [0x00007FF78E58E16B+3695259]
	GetHandleVerifier [0x00007FF78E5E6737+4057191]
	GetHandleVerifier [0x00007FF78E5DE4E3+4023827]
	GetHandleVerifier [0x00007FF78E2B04F9+689705]
	(No symbol) [0x00007FF78E18C048]
	(No symbol) [0x00007FF78E188044]
	(No symbol) [0x00007FF78E1881C9]
	(No symbol) [0x00007FF78E1788C4]
	BaseThreadInitThunk [0x00007FFC24088D17+23]
	RtlUserThreadStart [0x00007FFC243E7450+32]


In [37]:


# 初始化翻页次数计数器
page_count = 0
data = []

while True:
    try:
        # 尝试查找“下一页”按钮，这里使用了CSS选择器，您也可以根据实际情况使用XPath或其他选择器
        next_page_button = driver.find_element(By.CSS_SELECTOR, "li.ant-pagination-next[title='下一页']")
        # 检查“下一页”按钮是否被禁用（即已经到达最后一页）
        # 这一步是可选的，取决于页面的实际行为
        if 'ant-pagination-disabled' in next_page_button.get_attribute('class'):
            print("已到达最后一页。")
            break
        # 采集信息
        get_point_text_info(driver,data)
        # 点击“下一页”按钮
        next_page_button.click()
        
        # 增加翻页计数
        page_count += 1
        

        
    except NoSuchElementException:
        # 如果找不到“下一页”按钮，说明已经到达最后一页
        print("已经到达最后一页，无法继续翻页。")
        # 采集最后一页信息
        get_point_text_info(driver,data)
        break
    except Exception as e:
        # 处理其他可能的异常
        print(f"在翻页过程中发生错误：{e}")
        break

print(f"总共翻页次数：{page_count}")


已到达最后一页。
总共翻页次数：0


In [35]:
df = pd.DataFrame(data)

In [36]:
df

,样点编号,坐落单位名称,样点类别,采样类型,土壤类型,土地利用类型,是否水团样,是否盐碱地,布设位置经度,布设位置纬度,采样位置经度,采样位置纬度,是否修正,当前节点,调查时间,调查单位,调查人
0,5227010101000001,都匀市-平浪镇-甲壤村,表层样,普通样,水稻土_淹育水稻土_浅砂泥田_黄砂泥田,水田,否,否,107.235684,26.127202,107.235684,26.127202,否,待国家级审核,2023-10-20 16:46:27,上海数喆数据科技有限公司,浦世杰
1,5227010101000006,都匀市-毛尖镇-坪阳村,表层样,普通样,水稻土_潴育水稻土_潮泥田_潮板砂田,水田,否,否,107.352998,26.223011,107.352998,26.223011,否,待国家级审核,2023-10-27 16:25:21,上海数喆数据科技有限公司,郭洋
2,5227010101000008,都匀市-匀东镇-明英村,表层样,普通样,黄壤_典型黄壤_泥质黄壤_薄腐厚层泥质黄壤,水田,否,否,107.692013,26.201868,107.693136,26.202125,是,待国家级审核,2023-10-23 10:55:12,贵州省有色金属和核工业地质物化探总队,吴梦江
3,5227010101000011,都匀市-小围寨街道办事处-苗拱村,表层样,普通样,黄壤_典型黄壤_硅质黄壤_厚腐薄层硅质黄壤,水田,否,否,107.576772,26.080588,107.576772,26.080588,否,待国家级审核,2023-10-25 14:54:52,贵州省有色金属和核工业地质勘查局物化探总队,石明科
4,5227010101000013,都匀市-绿茵湖街道办事处-邦水村,表层样,普通样,水稻土_潴育水稻土_灰泥田_大眼黄泥田,水田,否,否,107.420482,26.291967,107.420482,26.291967,否,待国家级审核,2023-10-22 12:04:32,上海数喆数据科技有限公司,李国良
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,5227260301000471,独山县-百泉镇-摆罗村,表层样,普通样,水稻土_渗育水稻土_渗白粉泥田_黄渗白粉泥田,乔木林地,否,否,107.464225,25.839343,107.464225,25.839343,否,待国家级审核,2023-12-20 10:21:19,贵州师范大学,陈勇
2096,5227260301000506,独山县-麻尾镇-黄后村,表层样,普通样,石灰（岩）土_黑色石灰土_黑色石灰壤土_薄层黑色石灰壤土,乔木林地,否,否,107.533535,25.266463,107.533535,25.266463,否,待国家级审核,2023-12-28 09:26:57,贵州师范大学,杨福
2097,5227260301000543,独山县-基长镇-江寨村,表层样,普通样,黄壤_典型黄壤_硅质黄壤_薄腐薄层硅质黄壤,乔木林地,否,否,107.664355,25.846911,107.664355,25.846911,否,待国家级审核,2024-01-06 13:29:35,贵州师范大学,刘明华
2098,5227260301000586,独山县-麻尾镇-黄后村,表层样,普通样,水稻土_漂洗水稻土_漂白粉泥田_重漂白粉泥田,乔木林地,否,否,107.518868,25.304368,107.518868,25.304368,否,待国家级审核,2023-12-23 14:50:11,贵州师范大学,杨福


In [13]:
# 定位包含详情按钮的所有<tr>元素
rows = driver.find_elements(By.CSS_SELECTOR, "tr.ant-table-row")

In [14]:
rows

[<selenium.webdriver.remote.webelement.WebElement (session="81dcc4c857db69784c5ba40479a8f659", element="47D936E6B22A8C5CD85E86B5B9CFF231_element_214")>,
 <selenium.webdriver.remote.webelement.WebElement (session="81dcc4c857db69784c5ba40479a8f659", element="47D936E6B22A8C5CD85E86B5B9CFF231_element_215")>,
 <selenium.webdriver.remote.webelement.WebElement (session="81dcc4c857db69784c5ba40479a8f659", element="47D936E6B22A8C5CD85E86B5B9CFF231_element_216")>,
 <selenium.webdriver.remote.webelement.WebElement (session="81dcc4c857db69784c5ba40479a8f659", element="47D936E6B22A8C5CD85E86B5B9CFF231_element_217")>,
 <selenium.webdriver.remote.webelement.WebElement (session="81dcc4c857db69784c5ba40479a8f659", element="47D936E6B22A8C5CD85E86B5B9CFF231_element_218")>,
 <selenium.webdriver.remote.webelement.WebElement (session="81dcc4c857db69784c5ba40479a8f659", element="47D936E6B22A8C5CD85E86B5B9CFF231_element_219")>,
 <selenium.webdriver.remote.webelement.WebElement (session="81dcc4c857db69784c5ba4

In [ ]:
# 遍历每个<tr>元素
for row in rows:
    # 在每个<tr>元素中定位详情按钮
    detail_button = row.find_element(By.CSS_SELECTOR, "button.ant-btn.ant-btn-link.ant-btn-lg")
    
    # 点击详情按钮
    detail_button.click()
    
    # 处理详情页内容，可能需要等待内容加载
    time.sleep(2)  # 根据实际情况调整等待时间

    # 定位返回按钮
    back_button = driver.find_element(By.CSS_SELECTOR, "button.ant-btn")

    # 点击返回按钮
    back_button.click()


In [18]:
image_elements = driver.find_elements(By.TAG_NAME, "img")

In [ ]:
image_elements

In [ ]:
# 提取所有图片的src属性，即它们的链接
image_urls = [image.get_attribute('src') for image in image_elements]

# 输出图片链接
for url in image_urls:
    print(url)

In [ ]:
images_info = driver.execute_script("""
    var images = [];
    var imageElements = document.querySelectorAll('div.card-box div.ant-image img');
    imageElements.forEach(function(img) {
        var labelElement = img.nextElementSibling;
        var label = labelElement ? labelElement.innerText : '';
        images.push({'url': img.src, 'label': label});
    });
    return images;
""")
for info in images_info:
    print(info)


In [54]:
import re
html_source = driver.page_source
pattern = re.compile(r'<div class="ant-image"[^>]*><img[^>]+src="([^"]+)"[^>]*>.*?<div class="img-text"[^>]*>([^<]+)</div>', re.DOTALL)
matches = pattern.findall(html_source)
for match in matches:
    print({'url': match[0], 'label': match[1]})


In [ ]:

# 获取“下一张”按钮
next_button = driver.find_element(By.CSS_SELECTOR, 'div.swiper-button-next')

images_info = []
num_images = len(driver.find_elements(By.CSS_SELECTOR, 'div.card-box div.ant-image img'))

# 遍历图片集合
for _ in range(num_images):
    # 等待图片和标签信息更新
    time.sleep(1)  # 根据实际情况调整等待时间

    # 获取当前显示的图片URL和标签信息
    img = driver.find_element(By.CSS_SELECTOR, 'div.card-box div.ant-image img')
    labelDiv = driver.find_element(By.CSS_SELECTOR, "div.green-area")
    labelType = labelDiv.find_element(By.CSS_SELECTOR, "div.photo > div:first-child").text.replace('照片类型: ', '').strip()
    labelAngle = labelDiv.find_element(By.CSS_SELECTOR, "div.photo > div:last-child").text.replace('拍摄角度: ', '').strip()
    labelTime = labelDiv.find_element(By.CSS_SELECTOR, ".time-area").text.replace('拍摄时间: ', '').strip()

    images_info.append({
        'url': img.get_attribute('src'),
        'label': {'type': labelType, 'angle': labelAngle, 'time': labelTime}
    })

    # 点击“下一张”按钮
    try:
        next_button.click()
    except:
        pass

# 输出获取到的图片信息
# for info in images_info:
#     print(info)

In [95]:

# 等待并点击下拉选择的触发器以展开选项列表
dropdown_trigger = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.ant-select-selector'))
)
dropdown_trigger.click()

# 增加一些等待时间，确保下拉列表的选项加载完成

# 等待并定位到特定的选项（例如“混合点照片”）
option_to_select = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//div[contains(@class, 'ant-select-item-option-content') and contains(text(), '混样点照片')]"))
)
option_to_select.click()


In [98]:

# 定位所有包含文本的元素
text_elements = driver.find_elements(By.XPATH, '//*[text()]')

# # 提取并打印每个元素的文本
# for element in text_elements:
#     print(element.text)